In [149]:
import yfinance as yf
import pandas as pd
from tqdm import tqdm

# --------- Config ---------
TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', "PG", "MMM"]  # Replace with your 1000 tickers
YEARS = 4

# --------- Corrected Fields ---------
income_fields = ['Total Revenue', 'Operating Income', 'Pretax Income', 'Tax Provision']
cashflow_fields = ['Operating Cash Flow', 'Capital Expenditure', 'Free Cash Flow', 
                   'Depreciation And Amortization', 'Stock Based Compensation']
balance_fields = ['Current Assets', 'Current Liabilities', 'Cash And Cash Equivalents']

info_fields = ['beta', 'marketCap', 'sharesOutstanding', 'sector']

# --------- Logging lists ---------
missing_log = []

# --------- Core Extraction Function ---------

def get_financial_data(ticker_symbol):
    try:
        ticker = yf.Ticker(ticker_symbol)

        # Fetch financials
        income_stmt = ticker.financials.T.head(YEARS)
        cash_flow = ticker.cashflow.T.head(YEARS)
        balance_sheet = ticker.balance_sheet.T.head(YEARS)
        info = ticker.info

        # --- Income Statement ---
        for col in income_fields:
            if col not in income_stmt.columns:
                income_stmt[col] = pd.NA
                missing_log.append((ticker_symbol, 'Income Statement', col))
        income_df = income_stmt[income_fields].copy()
        income_df['Ticker'] = ticker_symbol

        # --- Cash Flow Statement ---
        for col in cashflow_fields:
            if col not in cash_flow.columns:
                cash_flow[col] = pd.NA
                missing_log.append((ticker_symbol, 'Cash Flow', col))
        cashflow_df = cash_flow[cashflow_fields].copy()
        cashflow_df['Ticker'] = ticker_symbol

        # --- Balance Sheet ---
        for col in balance_fields:
            if col not in balance_sheet.columns:
                balance_sheet[col] = pd.NA
                missing_log.append((ticker_symbol, 'Balance Sheet', col))
        balance_df = balance_sheet[balance_fields].copy()
        balance_df['Ticker'] = ticker_symbol

        # --- Info ---
        info_clean = {key: info.get(key, None) for key in info_fields}
        info_clean['Ticker'] = ticker_symbol
        info_df = pd.DataFrame([info_clean])

        return income_df, cashflow_df, balance_df, info_df

    except Exception as e:
        print(f"❌ Error with {ticker_symbol}: {e}")
        return None, None, None, None

# --------- Loop Through Tickers ---------

all_income = []
all_cashflow = []
all_balance = []
all_info = []

for t in tqdm(TICKERS):
    income, cashflow, balance, info = get_financial_data(t)
    if income is not None:
        all_income.append(income)
        all_cashflow.append(cashflow)
        all_balance.append(balance)
        all_info.append(info)

# --------- Concatenate Results ---------

df_income = pd.concat(all_income).reset_index().rename(columns={'index': 'Date'})
df_cashflow = pd.concat(all_cashflow).reset_index().rename(columns={'index': 'Date'})
df_balance = pd.concat(all_balance).reset_index().rename(columns={'index': 'Date'})
df_info = pd.concat(all_info).reset_index(drop=True)

# --------- Cleanup ---------
df_income['Date'] = pd.to_datetime(df_income['Date'])
df_cashflow['Date'] = pd.to_datetime(df_cashflow['Date'])
df_balance['Date'] = pd.to_datetime(df_balance['Date'])

df_income = df_income.round(0)
df_cashflow = df_cashflow.round(0)
df_balance = df_balance.round(0)



pd.DataFrame(missing_log, columns=['Ticker', 'Statement', 'Missing Column']).to_csv('missing_fields_log.csv', index=False)

print("✅ Done. Missing fields logged to 'missing_fields_log.csv'")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:02<00:00,  3.15it/s]

✅ Done. Missing fields logged to 'missing_fields_log.csv'


In [150]:
df_income

,Date,Total Revenue,Operating Income,Pretax Income,Tax Provision,Ticker
0,2024-09-30,3.910350e+11,1.232160e+11,1.234850e+11,2.974900e+10,AAPL
1,2023-09-30,3.832850e+11,1.143010e+11,1.137360e+11,1.674100e+10,AAPL
2,2022-09-30,3.943280e+11,1.194370e+11,1.191030e+11,1.930000e+10,AAPL
3,2021-09-30,3.658170e+11,1.089490e+11,1.092070e+11,1.452700e+10,AAPL
4,2025-06-30,2.817240e+11,1.285280e+11,1.236270e+11,2.179500e+10,MSFT
5,2024-06-30,2.451220e+11,1.094330e+11,1.077870e+11,1.965100e+10,MSFT
6,2023-06-30,2.119150e+11,8.852300e+10,8.931100e+10,1.695000e+10,MSFT
7,2022-06-30,1.982700e+11,8.338300e+10,8.371600e+10,1.097800e+10,MSFT
8,2024-12-31,3.500180e+11,1.123900e+11,1.198150e+11,1.969700e+10,GOOGL
9,2023-12-31,3.073940e+11,8.429300e+10,8.571700e+10,1.192200e+10,GOOGL


In [151]:
df_cashflow

,Date,Operating Cash Flow,Capital Expenditure,Free Cash Flow,Depreciation And Amortization,Stock Based Compensation,Ticker
0,2024-09-30,1.182540e+11,-9.447000e+09,1.088070e+11,1.144500e+10,1.168800e+10,AAPL
1,2023-09-30,1.105430e+11,-1.095900e+10,9.958400e+10,1.151900e+10,1.083300e+10,AAPL
2,2022-09-30,1.221510e+11,-1.070800e+10,1.114430e+11,1.110400e+10,9.038000e+09,AAPL
3,2021-09-30,1.040380e+11,-1.108500e+10,9.295300e+10,1.128400e+10,7.906000e+09,AAPL
4,2025-06-30,1.361620e+11,-6.455100e+10,7.161100e+10,3.415300e+10,1.197400e+10,MSFT
5,2024-06-30,1.185480e+11,-4.447700e+10,7.407100e+10,2.228700e+10,1.073400e+10,MSFT
6,2023-06-30,8.758200e+10,-2.810700e+10,5.947500e+10,1.386100e+10,9.611000e+09,MSFT
7,2022-06-30,8.903500e+10,-2.388600e+10,6.514900e+10,1.446000e+10,7.502000e+09,MSFT
8,2024-12-31,1.252990e+11,-5.253500e+10,7.276400e+10,1.531100e+10,2.278500e+10,GOOGL
9,2023-12-31,1.017460e+11,-3.225100e+10,6.949500e+10,1.194600e+10,2.246000e+10,GOOGL


In [152]:
df_balance

,Date,Current Assets,Current Liabilities,Cash And Cash Equivalents,Ticker
0,2024-09-30,1.529870e+11,1.763920e+11,2.994300e+10,AAPL
1,2023-09-30,1.435660e+11,1.453080e+11,2.996500e+10,AAPL
2,2022-09-30,1.354050e+11,1.539820e+11,2.364600e+10,AAPL
3,2021-09-30,1.348360e+11,1.254810e+11,3.494000e+10,AAPL
4,2025-06-30,1.911310e+11,1.412180e+11,3.024200e+10,MSFT
5,2024-06-30,1.597340e+11,1.252860e+11,1.831500e+10,MSFT
6,2023-06-30,1.842570e+11,1.041490e+11,3.470400e+10,MSFT
7,2022-06-30,1.696840e+11,9.508200e+10,1.393100e+10,MSFT
8,2024-12-31,1.637110e+11,8.912200e+10,2.346600e+10,GOOGL
9,2023-12-31,1.715300e+11,8.181400e+10,2.404800e+10,GOOGL


In [89]:
# caluclate present FCFF, Use 4 year average to find growth rate for next three years
# Terminal value calcualtion after 3 years, I need to find the WACC? Estimate or ML? First estimate, ML as an add on later.

In [153]:
def compute_historical_fcff_all_tickers(df_income, df_cashflow, df_balance):
    # Ensure date formats are datetime
    df_income['Date'] = pd.to_datetime(df_income['Date'])
    df_cashflow['Date'] = pd.to_datetime(df_cashflow['Date'])
    df_balance['Date'] = pd.to_datetime(df_balance['Date'])

    # Merge all 3 DataFrames on Date + Ticker
    df = df_income.merge(df_cashflow, on=['Date', 'Ticker'], how='inner')\
                  .merge(df_balance, on=['Date', 'Ticker'], how='inner')

    # Sort for diff calculation
    df = df.sort_values(by=['Ticker', 'Date'])

    # Compute Tax Rate safely
    df['Tax Rate'] = df['Tax Provision'] / df['Pretax Income']
    df['Tax Rate'] = df['Tax Rate'].clip(lower=0, upper=1).fillna(0.21)

    # Compute NWC and ΔNWC per Ticker
    df['NWC'] = df['Current Assets'] - df['Current Liabilities']
    df['ΔNWC'] = df.groupby('Ticker')['NWC'].diff()

    # Compute FCFF
    df['FCFF'] = (
        df['Operating Income'] * (1 - df['Tax Rate']) +
        df['Depreciation And Amortization'] -
        df['Capital Expenditure'] -
        df['ΔNWC']
    )

    # Select clean output
    fcff_df = df[['Date', 'Ticker', 'FCFF', 'Operating Income', 'Tax Rate',
                  'Depreciation And Amortization', 'Capital Expenditure', 'ΔNWC']].copy()

    # Optional rounding
    fcff_df = fcff_df.round(2)

    return fcff_df

In [154]:
fcff_df = compute_historical_fcff_all_tickers(df_income, df_cashflow, df_balance)
fcff_df = fcff_df[fcff_df['FCFF'].notna()]
fcff_df  # or display(fcff_df) in Jupyter

,Date,Ticker,FCFF,Operating Income,Tax Rate,Depreciation And Amortization,Capital Expenditure,ΔNWC
2,2022-09-30,AAPL,1.498269e+11,1.194370e+11,0.16,1.110400e+10,-1.070800e+10,-2.793200e+10
1,2023-09-30,AAPL,1.031198e+11,1.143010e+11,0.15,1.151900e+10,-1.095900e+10,1.683500e+10
0,2024-09-30,AAPL,1.360868e+11,1.232160e+11,0.24,1.144500e+10,-9.447000e+09,-2.166300e+10
14,2022-12-31,AMZN,1.390922e+11,1.224800e+10,0.54,4.192100e+10,-6.364500e+10,-2.791600e+10
13,2023-12-31,AMZN,1.152217e+11,3.685200e+10,0.19,4.866300e+10,-5.272900e+10,1.603600e+10
12,2024-12-31,AMZN,1.911228e+11,6.859300e+10,0.14,5.279500e+10,-8.299900e+10,4.002000e+09
10,2022-12-31,GOOGL,1.362805e+11,7.484200e+10,0.16,1.347500e+10,-3.148500e+10,-2.839400e+10
9,2023-12-31,GOOGL,1.225451e+11,8.429300e+10,0.14,1.194600e+10,-3.225100e+10,-5.779000e+09
8,2024-12-31,GOOGL,1.768866e+11,1.123900e+11,0.16,1.531100e+10,-5.253500e+10,-1.512700e+10
18,2022-12-31,META,7.618063e+10,2.894400e+10,0.19,8.686000e+09,-3.118600e+10,-1.300800e+10


In [155]:
def project_fcff_from_history(fcff_df, projection_years=3):
    projections_fcff = []

    # Growth caps per industry (you can tweak these)
    industry_growth_cap = {
        'Technology': 0.9,
        'Consumer Defensive': 0.06,
        'Consumer Cyclical': 0.07,
        'Healthcare': 0.08,
        'Industrials': 0.05,
        'Financial Services': 0.06,
        'Energy': 0.04,
        'Utilities': 0.03,
        'Basic Materials': 0.05,
        'Real Estate': 0.04,
        'Communication Services': 0.08
    }

    # Cache industry lookups to avoid redundant API calls
    industry_cache = {}

    def get_industry(ticker):
        if ticker in industry_cache:
            return industry_cache[ticker]
        try:
            info = yf.Ticker(ticker).info
            industry = info.get('sector', 'Unknown')
        except Exception:
            industry = 'Unknown'
        industry_cache[ticker] = industry
        return industry

    for ticker in fcff_df['Ticker'].unique():
        ticker_df = fcff_df[fcff_df['Ticker'] == ticker].sort_values('Date').tail(3)

        if len(ticker_df) < 3:
            continue

        fcff_vals = ticker_df['FCFF'].values

        # Avoid division by zero or erratic values
        if fcff_vals[0] == 0 or fcff_vals[1] == 0 or any(pd.isna(fcff_vals)):
            continue

        g1 = (fcff_vals[1] - fcff_vals[0]) / fcff_vals[0]
        g2 = (fcff_vals[2] - fcff_vals[1]) / fcff_vals[1]
        avg_growth = (g1 + g2) / 2

        # Lookup industry and apply cap
        industry = get_industry(ticker)
        cap = industry_growth_cap.get(industry, 0.06)  # fallback cap
        avg_growth = max(min(avg_growth, cap), 0.00)   # apply cap and floor

        last_fcff = fcff_vals[-1]
        last_year = pd.to_datetime(ticker_df['Date'].max()).year

        for i in range(1, projection_years + 1):
            future_year = last_year + i
            projected_fcff = last_fcff * ((1 + avg_growth) ** i)

            projections_fcff.append({
                'Ticker': ticker,
                'Year': future_year,
                'Projected FCFF': round(projected_fcff, 2),
                'Growth Rate': round(avg_growth, 4),
                'Industry': industry,
                'Cap Used': cap
            })

    return pd.DataFrame(projections_fcff)

In [156]:
projections_df = project_fcff_from_history(fcff_df, projection_years=3)
display(projections_df)

,Ticker,Year,Projected FCFF,Growth Rate,Industry,Cap Used
0,AAPL,2025,1.366281e+11,0.0040,Technology,0.90
1,AAPL,2026,1.371716e+11,0.0040,Technology,0.90
2,AAPL,2027,1.377173e+11,0.0040,Technology,0.90
3,AMZN,2025,2.045014e+11,0.0700,Consumer Cyclical,0.07
4,AMZN,2026,2.188165e+11,0.0700,Consumer Cyclical,0.07
5,AMZN,2027,2.341337e+11,0.0700,Consumer Cyclical,0.07
6,GOOGL,2025,1.910376e+11,0.0800,Communication Services,0.08
7,GOOGL,2026,2.063206e+11,0.0800,Communication Services,0.08
8,GOOGL,2027,2.228262e+11,0.0800,Communication Services,0.08
9,META,2025,1.090128e+11,0.0800,Communication Services,0.08


In [157]:
# WACC = CAPM? Cost of Debt?

In [158]:
# CAPM

In [159]:

tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META']  # Replace with your 1000 tickers

def compute_capm_for_tickers(tickers, risk_free_rate=0.042, market_return=0.09):
    """
    Computes CAPM (Cost of Equity) for a list of tickers using beta from yfinance.
    """
    capm_results = []

    for ticker in tickers:
        try:
            yf_ticker = yf.Ticker(ticker)
            beta = yf_ticker.info.get('beta', None)

            if beta is not None:
                cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)
            else:
                cost_of_equity = None

            capm_results.append({
                'Ticker': ticker,
                'Beta': round(beta, 3) if beta else None,
                'Cost of Equity (CAPM)': round(cost_of_equity, 4) if cost_of_equity else None
            })

        except Exception as e:
            capm_results.append({
                'Ticker': ticker,
                'Error': str(e)
            })

    return pd.DataFrame(capm_results)

In [160]:
compute_capm_for_tickers(TICKERS, risk_free_rate=0.042, market_return=0.09)

,Ticker,Beta,Cost of Equity (CAPM)
0,AAPL,1.165,0.0979
1,MSFT,1.055,0.0926
2,GOOGL,1.014,0.0907
3,AMZN,1.314,0.1051
4,META,1.273,0.1031
5,PG,0.373,0.0599
6,MMM,1.104,0.0950


In [161]:
# Cost of Debt

In [162]:
import yfinance as yf
import pandas as pd

def compute_cost_of_debt(ticker_symbol, tax_rate=None):
    """
    Computes cost of debt using:
    Cost of Debt = |Interest Expense| / Total Debt (from yfinance)
    Optionally returns after-tax cost of debt if tax_rate is provided.
    """
    try:
        ticker = yf.Ticker(ticker_symbol)

        # Get Income Statement and Balance Sheet
        income_stmt = ticker.income_stmt.T
        balance_sheet = ticker.balance_sheet.T

        # Pull relevant data
        interest_expense = income_stmt.get('Interest Expense')
        total_debt = balance_sheet.get('Total Debt')

        # Handle missing data
        if interest_expense is None or total_debt is None:
            print(f"Missing data for {ticker_symbol}")
            return None

        # Use latest non-null values
        latest_interest = interest_expense.dropna().iloc[-1]
        latest_debt = total_debt.dropna().iloc[-1]

        if latest_debt == 0:
            return None  # avoid division by zero

        raw_cost = abs(latest_interest) / latest_debt
        cost_of_debt = round(raw_cost * (1 - tax_rate), 4) if tax_rate else round(raw_cost, 4)

        return cost_of_debt

    except Exception as e:
        print(f"Error for {ticker_symbol}: {e}")
        return None

In [163]:
results = []
for t in TICKERS:
    cod = compute_cost_of_debt(t, tax_rate=0.21)  # ← After-tax cost
    results.append({'Ticker': t, 'Cost of Debt': cod})

cost_of_debt_df = pd.DataFrame(results)
display(cost_of_debt_df)

,Ticker,Cost of Debt
0,AAPL,0.0166
1,MSFT,0.0266
2,GOOGL,0.0096
3,AMZN,0.0123
4,META,0.0013
5,PG,0.0107
6,MMM,0.0211


In [164]:
# WACC 

In [165]:
capm_df = compute_capm_for_tickers(TICKERS, risk_free_rate=0.042, market_return=0.09)
capm_dict = capm_df.set_index('Ticker')['Cost of Equity (CAPM)'].to_dict()

def compute_wacc_using_existing_functions(ticker_symbol, capm_dict, tax_rate=0.21):
    try:
        ticker = yf.Ticker(ticker_symbol)

        # Cost of Equity from CAPM results
        cost_of_equity = capm_dict.get(ticker_symbol)
        if cost_of_equity is None:
            print(f"Missing CAPM for {ticker_symbol}")
            return None

        # Cost of Debt using your function
        cost_of_debt = compute_cost_of_debt(ticker_symbol, tax_rate=tax_rate)
        if cost_of_debt is None:
            print(f"Missing Cost of Debt for {ticker_symbol}")
            return None

        # Capital structure
        info = ticker.info
        equity = info.get('marketCap', None)
        balance_sheet = ticker.balance_sheet.T
        total_debt_series = balance_sheet.get('Total Debt')

        if equity is None or total_debt_series is None:
            print(f"Missing capital data for {ticker_symbol}")
            return None

        debt = total_debt_series.dropna().iloc[-1]
        total_value = equity + debt
        if total_value == 0:
            return None

        # Weighted Average Cost of Capital
        wacc = (equity / total_value) * cost_of_equity + (debt / total_value) * cost_of_debt
        return round(wacc, 4)

    except Exception as e:
        print(f"Error for {ticker_symbol}: {e}")
        return None


results = []

for t in TICKERS:
    wacc = compute_wacc_using_existing_functions(t, capm_dict)
    results.append({'Ticker': t, 'WACC': wacc})

wacc_df = pd.DataFrame(results)
display(wacc_df)

,Ticker,WACC
0,AAPL,0.0945
1,MSFT,0.0916
2,GOOGL,0.0897
3,AMZN,0.1008
4,META,0.1024
5,PG,0.0558
6,MMM,0.0813


In [166]:
# DCF

In [167]:
def run_dcf_for_tickers(tickers, fcff_df, tax_rate=0.21, risk_free_rate=0.042, market_return=0.09, projection_years=3, terminal_growth_rate=0.02):
    # Step 1: Compute cost of equity via CAPM
    capm_df = compute_capm_for_tickers(TICKERS, risk_free_rate, market_return)
    capm_dict = capm_df.set_index('Ticker')['Cost of Equity (CAPM)'].to_dict()
    
    # Step 2: Compute WACC per ticker
    wacc_results = []
    for t in tickers:
        wacc = compute_wacc_using_existing_functions(t, capm_dict, tax_rate)
        wacc_results.append({'Ticker': t, 'WACC': wacc})
    wacc_df = pd.DataFrame(wacc_results)

    # Step 3: Project FCFF
    projections_df = project_fcff_from_history(fcff_df, projection_years=projection_years)

    # Step 4: Calculate DCF using everything above
    dcf_results = []

    for ticker in tickers:
        # Get WACC
        wacc_row = wacc_df[wacc_df['Ticker'] == ticker]
        if wacc_row.empty:
            continue
        wacc = wacc_row['WACC'].values[0]
        if wacc is None or wacc == 0:
            continue

        # Get FCFF projections
        proj_rows = projections_df[projections_df['Ticker'] == ticker].sort_values('Year')
        projected_fcffs = proj_rows['Projected FCFF'].values

        if len(projected_fcffs) == 0:
            continue

        # Discount projected FCFFs
        discounted_fcffs = [fcff / ((1 + wacc) ** (i + 1)) for i, fcff in enumerate(projected_fcffs)]

        # Terminal value
        last_fcff = projected_fcffs[-1]
        terminal_value = (last_fcff * (1 + terminal_growth_rate)) / (wacc - terminal_growth_rate)
        discounted_terminal_value = terminal_value / ((1 + wacc) ** len(projected_fcffs))

        # Combine for enterprise value
        dcf_value = sum(discounted_fcffs) + discounted_terminal_value

        dcf_results.append({
            'Ticker': ticker,
            'DCF Value': round(dcf_value, 2),
            'WACC': round(wacc, 4),
            'Growth Rate': round(proj_rows['Growth Rate'].values[0], 4)
        })
    print(f"TV % of DCF: {discounted_terminal_value / (sum(discounted_fcffs) + discounted_terminal_value):.2%}")
    return pd.DataFrame(dcf_results)

In [168]:
dcf_df = run_dcf_for_tickers(TICKERS, fcff_df)
display(dcf_df)

TV % of DCF: 83.66%


,Ticker,DCF Value,WACC,Growth Rate
0,AAPL,1.782462e+12,0.0945,0.0040
1,MSFT,6.655675e+12,0.0916,0.4015
2,GOOGL,3.041340e+12,0.0897,0.0800
3,AMZN,2.757662e+12,0.1008,0.0700
4,META,1.465517e+12,0.1024,0.0800
5,PG,7.855674e+11,0.0558,0.0600
6,MMM,4.744115e+10,0.0813,0.0000
